In [76]:
# https://docs.binance.us/?python#get-candlestick-data
# binance.us
# Github.com - send Daniel Github username
# https://algotrading101.com/learn/binance-python-api-guide/
# https://data.binance.vision/
# https://github.com/sammchardy/python-binance

"""
Max requests per minute: 1200 (then IP ban)
Python-binance purposefully implements delays to prevent IP ban due to overloading
Some code is asyncronous
"""
from datetime import datetime
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import requests
from time import sleep
from datetime import datetime, timedelta
import time

import sys
sys.path.append('../../')
from src.PaperTrader import PaperTrader
from src.TechAnalysis import TechAnalysis

from config import key, secret_key
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException
from mysql.connector import connect
connect.raise_on_warnings = False
from db_config import host, port, username, password
# from preprocess import main

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score
from xgboost import XGBClassifier
import pickle

client = Client(key, secret_key, tld='us') # Binance
connection = connect(host=host,
        user=username,
        password=password)
cursor = connection.cursor() # SQL

resp = requests.get('https://api.binance.us/api/v3/ping')
# resp = requests.get('https://api.binance.us/api/v3/klines?symbol=BTCUSD&interval=1m&limit=1000')
# print(pd.DataFrame(resp.json(), columns = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset','taker_buy_quote_asset_volume', 'ignore']))

In [77]:
def to_unixmillis(from_date):
	fmt = "%Y-%m-%d %H:%M:%S"
	from_date_obj = datetime.strptime(from_date, fmt)
	past = datetime(1970, 1, 1, tzinfo=from_date_obj.tzinfo)
	return int((from_date_obj - past).total_seconds() * 1000.0)


def to_datetime(ms):
	return datetime.fromtimestamp(int(float(ms) / 1000.0))

def to_python_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.fromtimestamp(timestamp)

"""
BNB Coin allows for reduced commission fees. 25% cheaper
"""

def topup_bnb(min_balance: float, topup: float):
	''' Top up BNB balance if it drops below minimum specified balance '''
	bnb_balance = client.get_asset_balance(asset='BNB')
	bnb_balance = float(bnb_balance['free'])
	if bnb_balance < min_balance:
		qty = round(topup - bnb_balance, 5)
		print(qty)
		order = client.order_market_buy(symbol='BNBUSDT', quantity=qty)
		return order
	return False

In [78]:
# print(client.get_asset_balance(asset='BTC'))
# print(client.get_asset_balance(asset='USD'))
# get market depth
# depth = client.get_order_book(symbol='BTCUSD')

# place a test market buy order, to place an actual order use the create_order function
# order = client.create_test_order(
#     symbol='BNBBTC',
#     side=Client.SIDE_BUY,
#     type=Client.ORDER_TYPE_MARKET,
#     quantity=100)

# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
eth_price = client.get_symbol_ticker(symbol="ETHUSDT")
conv_price = client.get_symbol_ticker(symbol="ETHBTC")
# print full output (dictionary)
print(btc_price)
print(eth_price)
print(conv_price)
print(client.get_asset_balance(asset='BNB'))

{'symbol': 'BTCUSDT', 'price': '39358.43000000'}
{'symbol': 'ETHUSDT', 'price': '2934.13000000'}
{'symbol': 'ETHBTC', 'price': '0.07454600'}
{'asset': 'BNB', 'free': '0.04793826', 'locked': '0.00000000'}


In [79]:
# from time import sleep
# from datetime import datetime, timedelta
# import time
# from binance import ThreadedWebsocketManager

# btc_price = {'error':False}
# def btc_trade_history(msg):
#     ''' define how to process incoming WebSocket messages '''
#     if msg['e'] != 'error':
#         print(msg['c'])
#         btc_price['last'] = msg['c']
#         btc_price['bid'] = msg['b']
#         btc_price['last'] = msg['a']
#         btc_price['error'] = False
#     else:
#         btc_price['error'] = True

# bsm = ThreadedWebsocketManager()
# bsm.start()
# bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='BTCUSDT')
# bsm.stop()

In [80]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

def binance_data_to_sql(start_date, end_date, currencytype, timespan, connection):
    cursor = connection.cursor()
    client = Client(key, secret_key, tld='us') # Binance
    try:
        cursor.execute(f"""create table AlgoModule.{currencytype}_{timespan}
        (
        open_time datetime,
        close_time datetime,
        open_price double,
        close_price double,
        high_price double, 
        low_price double,
        volume double,
        quote_av double,
        trades int,
        tb_base_av double,
        tb_quote_av double
        )""")
        connection.commit()
    except:
        print("Error, but moving on")
        
    step_date = start_date
    while step_date <= end_date:
        print(f'pulling {step_date} data for AlgoModule.{currencytype}_{timespan}')
        bars = client.get_historical_klines(currencytype, timespan, start_str = f'{step_date}', end_str=f'{step_date + timedelta(hours=12)}', limit=1000)
        bars = pd.DataFrame(bars, columns = ['date', 'open',
                    'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
                    'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
        bars['date'] = bars['date'].apply(to_datetime)
        bars['close_time'] = bars['close_time'].apply(to_datetime)
        for _, row in bars.iterrows():
            cursor.execute(f"""INSERT INTO AlgoModule.{currencytype}_{timespan} (open_time, close_time, open_price, close_price, high_price, low_price,
            volume, quote_av, trades, tb_base_av, tb_quote_av) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
            , (row.date, row.close_time, row.open, row.close, row.high, row.low, row.volume, row.quote_av, row.trades, row.tb_base_av, row.tb_quote_av))
        step_date = step_date + timedelta(hours=12)
        connection.commit()
        sleep(1)


def run():
    connection = connect(host=host,
        user=username,
        password=password)
    # get timestamp of earliest date data is available # fmt = "%Y-%m-%d %H:%M:%
    start_date = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
    start_date = to_datetime(start_date)
    end_date = datetime.now()

    currency_pairings = [ 'ETHBTC']
    # 'ETHUSD',
    #  'BTCUSD']
    time_intervals = [
    '1m',
    '3m',
     '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h']
    for cur in currency_pairings:
        for t in time_intervals:
            binance_data_to_sql(start_date, end_date, cur, t, connection)



def update_tables():
    connection = connect(host=host,
        user=username,
        password=password)
    end_date = datetime.now()
    currency_pairings = ['ETHUSD', 'BTCUSD', 'ETHBTC']
    time_intervals = ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h']
        
    for cur in currency_pairings:
        for t in time_intervals:
                try:
                    start_date = pd.read_sql(f"""select max(open_time) from AlgoModule.{cur}_{t}""", connection).values[0][0]
                    start_date = to_python_datetime(start_date)
                    # print(cur, t, start_date)          
                    binance_data_to_sql(start_date, end_date, cur, t, connection)
                except:
                    print(f'AlgoModule.{cur}_{t} does not exist')

In [8]:
update_tables()


Error, but moving on
pulling 2022-04-17 15:54:00 data for AlgoModule.ETHUSD_1m
Error, but moving on
pulling 2022-04-15 08:06:00 data for AlgoModule.ETHUSD_3m
pulling 2022-04-15 20:06:00 data for AlgoModule.ETHUSD_3m
pulling 2022-04-16 08:06:00 data for AlgoModule.ETHUSD_3m
pulling 2022-04-16 20:06:00 data for AlgoModule.ETHUSD_3m
pulling 2022-04-17 08:06:00 data for AlgoModule.ETHUSD_3m
Error, but moving on
pulling 2022-04-13 05:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-13 17:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-14 05:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-14 17:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-15 05:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-15 17:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-16 05:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-16 17:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-17 05:20:00 data for AlgoModule.ETHUSD_5m
pulling 2022-04-17 17:20:00 data for AlgoModule.ETHUSD_5m
Error, bu

In [112]:
def preprocessing(df, price_offset = 1.0015, prediction = False, best_d_value = 1, len_of_original_weights = 64):
    column_name = 'close_price'
    ta = TechAnalysis(df)
    """
    Imputes fractional differencing into data
    """
    if prediction:
        df = df[(len(df) - (len_of_original_weights+1)):]
        df_fd, weights = ta.frac_diff(df[column_name], best_d_value)        
    else:
        
        df_fd, weights, best_d_value = ta.fractional_difference(column_name, alpha=.05)
          
    df['frac_diff_cost'] = np.nan
    df['frac_diff_cost'].iloc[len(weights):] = df_fd[0]

    """
    Create mass features
    """
    steps = [5, 10, 20, 30, 40, 50]
    macds = [[2,10],[5,10],[10,20],[10,30],[20,30]]
    bbs_std = [1, 1.5, 2]

    for step in steps:
        df[f'ma_{step}'] = ta.moving_average(column_name, step)
        df[f'ewa_{step}'] = ta.moving_average(column_name, step, simple=False)
        df[f'rsi_{step}'] = ta.rsi(column_name, step)
        for std in bbs_std:
            df[f'bb_{step}_{std}_upper'],  df[f'bb_{step}_{std}_lower']= ta.bollinger_bands(column_name, step, std = std)


    for macd in macds:
        short, long = macd
        df[f'rsi_{step}'] = ta.macd(column_name, short, long)

    if not prediction:
        df['target_classifier'] = 0
        df['target_classifier'][df['target']>df['close_price']*price_offset] = 1
        df.reset_index(inplace=True, drop=True)
        df.drop(['target'], axis=1, inplace=True)
        
    df.drop(['open_time', 'close_time','open_price', 'high_price', 'low_price'], axis=1, inplace=True)
    df.dropna(inplace=True)
    
    return df, best_d_value, weights

In [113]:
data = pd.read_sql("""select distinct * from AlgoModule.BTCUSD_1h""", connection)
data['target'] = data['close_price'].shift(-1)
data = data[(data['open_time'] > '2022-01-01 07:50:00') & (data['open_time'] < datetime.now())]

data, best_d_value, weights = preprocessing(data, 1.0015)
# ta = TechAnalysis(data)
# price_offset = 1.001
# column_name = 'close_price'
# df_fd, weights, best_d_value = ta.fractional_difference(column_name, alpha=.05)
# data['frac_diff_cost'] = np.nan
# data['frac_diff_cost'].iloc[len(weights):] = df_fd[0]

# steps = [5, 10, 20, 30, 40, 50]
# macds = [[2,10],[5,10],[10,20],[10,30],[20,30]]
# bbs_std = [1, 1.5, 2]

# for step in steps:
#     data[f'ma_{step}'] = ta.moving_average(column_name, step)
#     data[f'ewa_{step}'] = ta.moving_average(column_name, step, simple=False)
#     data[f'rsi_{step}'] = ta.rsi(column_name, step)
#     for std in bbs_std:
#         data[f'bb_{step}_{std}_upper'],  data[f'bb_{step}_{std}_lower']= ta.bollinger_bands(column_name, step, std = std)


# for macd in macds:
#     short, long = macd
#     data[f'rsi_{step}'] = ta.macd(column_name, short, long)

# data['target_classifier'] = 0
# data['target_classifier'][data['target']>data['close_price']*price_offset] = 1
# data.reset_index(inplace=True, drop=True)
# data.drop(['target'], axis=1, inplace=True)
    
# data.drop(['open_time', 'close_time','open_price', 'high_price', 'low_price'], axis=1, inplace=True)
# data.dropna(inplace=True)

In [114]:
data

,close_price,volume,quote_av,trades,tb_base_av,tb_quote_av,frac_diff_cost,ma_5,ewa_5,rsi_5,...,ma_50,ewa_50,rsi_50,bb_50_1_upper,bb_50_1_lower,bb_50_1.5_upper,bb_50_1.5_lower,bb_50_2_upper,bb_50_2_lower,target_classifier
49,46878.30,74.786196,3.512245e+06,1731,33.620991,1.579117e+06,36126.336382,47153.992,46981.747750,44.208295,...,47210.1152,46981.747750,77.558500,47444.197922,46976.032478,47561.239283,46858.991117,47678.280644,46741.949756,0
50,46689.17,48.715227,2.280845e+06,1817,18.996862,8.900235e+05,35836.917894,47063.372,46786.695917,25.654445,...,47203.1614,46786.695917,70.687500,47447.438346,46958.884454,47569.576819,46836.745981,47691.715292,46714.607508,0
51,46478.70,67.969090,3.159954e+06,2795,32.152553,1.495162e+06,35665.928086,46882.912,46581.365306,0.000000,...,47194.1832,46581.365306,74.023667,47456.386402,46931.979998,47587.488003,46800.878397,47718.589603,46669.776797,1
52,46593.98,47.675761,2.222376e+06,1776,30.276663,1.411300e+06,35475.228890,46761.864,46589.775102,13.792773,...,47181.4962,46589.775102,73.894667,47457.021869,46905.970531,47594.784703,46768.207697,47732.547537,46630.444863,0
53,46481.77,26.582900,1.238388e+06,1373,9.156700,4.265045e+05,35613.698326,46624.384,46517.771701,12.558281,...,47165.2970,46517.771701,85.621000,47457.514066,46873.079934,47603.622599,46726.971401,47749.731132,46580.862868,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,40069.94,30.778356,1.235268e+06,748,13.593420,5.455255e+05,30787.833940,40098.000,40093.539576,44.545002,...,40303.6484,40093.539576,86.166500,41079.939612,39527.357188,41468.085218,39139.211582,41856.230824,38751.065976,0
2440,39809.29,48.424642,1.932353e+06,1457,23.529697,9.391362e+05,30686.263272,40024.968,39904.039859,26.589907,...,40253.8756,39904.039859,92.273833,40977.699612,39530.051588,41339.611618,39168.139582,41701.523623,38806.227577,0
2441,39717.81,31.457964,1.254222e+06,1175,15.010662,5.985600e+05,30429.802905,39976.386,39779.886620,31.532532,...,40203.2212,39779.886620,103.202333,40870.909611,39535.532789,41204.753817,39201.688583,41538.598023,38867.844377,0
2442,39714.90,67.193329,2.673364e+06,1797,31.549799,1.255403e+06,30356.109563,39896.008,39736.562207,10.170264,...,40151.1976,39736.562207,115.296667,40748.530882,39553.864318,41047.197523,39255.197677,41345.864165,38956.531035,1


In [115]:
# df_fd
data['target_classifier'], data['target_classifier'].sum()
# weights
# best_d_value


(49      0
 50      0
 51      1
 52      0
 53      0
        ..
 2439    0
 2440    0
 2441    0
 2442    1
 2443    0
 Name: target_classifier, Length: 2395, dtype: int64,
 890)

In [116]:
"""
test/train/ver split
"""
train_size = round(.9*len(data))

train = data[:train_size]
test = data[train_size:]

y_train = train.pop('target_classifier')
X_train = train
# X_train.drop('datetime', axis=1, inplace=True)

y_test = test.pop('target_classifier')
X_test = test


In [117]:
bestfeatures = SelectKBest(score_func=f_regression, k=10)
fit = bestfeatures.fit(X_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)
featureScores = pd.concat([dfcolumns, dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

            Specs      Score
9           rsi_5  13.351254
0     close_price   8.080055
15   bb_5_2_lower   7.777138
8           ewa_5   7.597166
17         ewa_10   7.597166
26         ewa_20   7.597166
35         ewa_30   7.597166
44         ewa_40   7.597166
53         ewa_50   7.597166
24  bb_10_2_lower   7.595363


In [125]:
"""
ML Pipeline
with Verification
"""
pipe = make_pipeline(SelectKBest(score_func=f_regression, k=10),  GradientBoostingClassifier())
# StandardScaler(),
# SelectKBest(score_func=f_regression, k=10),
tscv = TimeSeriesSplit(n_splits=10)
parameters = {
    'gradientboostingclassifier__max_depth': range (2, 20, 2),
    'gradientboostingclassifier__n_estimators': range(60, 220, 40),
    # 'gradientboostingclassifier__n_estimators': [140],
    'gradientboostingclassifier__learning_rate': [0.1, 0.01, 0.05, .001]
}
# clf = GridSearchCV(pipe, parameters, n_jobs=-2, cv=tscv, scoring = 'precision')
clf = GridSearchCV(pipe, parameters, n_jobs=-1, cv=tscv, scoring = 'roc_auc')
clf.fit(X_train,y_train)
clf.best_params_

# load
# with open('model/model.pkl', 'rb') as f:
#     clf = pickle.load(f)
    

{'gradientboostingclassifier__learning_rate': 0.01,
 'gradientboostingclassifier__max_depth': 2,
 'gradientboostingclassifier__n_estimators': 100}

In [119]:
"""
save
"""
# with open('model/model.pkl','wb') as f:
#     pickle.dump(clf,f)

'\nsave\n'

In [120]:
# print(clf.cv_results_)
print(f"best score {clf.best_score_}")
print(clf.best_estimator_)
y_pred_train = clf.predict(X_train)
print(precision_score(y_train, y_pred_train), accuracy_score(y_train, y_pred_train))

y_pred_test = clf.predict(X_test)
print(precision_score(y_test, y_pred_test), accuracy_score(y_test, y_pred_test))



best score 0.4175290435214034
Pipeline(steps=[('selectkbest',
                 SelectKBest(score_func=<function f_regression at 0x000002A50D837670>)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_depth=8, n_estimators=60))])
1.0 0.9281076066790352
0.3114754098360656 0.5774058577405857


In [122]:
X_test

,close_price,volume,quote_av,trades,tb_base_av,tb_quote_av,frac_diff_cost,ma_5,ewa_5,rsi_5,...,bb_40_2_lower,ma_50,ewa_50,rsi_50,bb_50_1_upper,bb_50_1_lower,bb_50_1.5_upper,bb_50_1.5_lower,bb_50_2_upper,bb_50_2_lower
2205,46199.83,5.038555,2.330197e+05,573,2.169520,1.003329e+05,35561.175546,46346.210,46258.334199,33.669388,...,45822.098436,46254.9576,46258.334199,154.535833,46686.397302,45823.517898,46902.117153,45607.798047,47117.837004,45392.078196
2206,46263.47,4.957721,2.292112e+05,479,3.324556,1.537372e+05,35355.081151,46307.690,46261.758066,31.732994,...,45817.924243,46278.5436,46261.758066,147.323000,46675.531866,45881.555334,46874.025999,45683.061201,47072.520132,45484.567068
2207,46407.32,3.901355,1.808158e+05,627,1.851769,8.585417e+04,35427.000016,46313.102,46358.799355,52.273109,...,45827.709822,46301.6472,46358.799355,135.123000,46670.284367,45933.010033,46854.602950,45748.691450,47038.921534,45564.372866
2208,46459.81,16.165343,7.519424e+05,1028,9.940534,4.625826e+05,35570.584798,46345.940,46426.139785,64.579892,...,45831.097218,46329.0076,46426.139785,126.131833,46654.226714,46003.788486,46816.836271,45841.178929,46979.445828,45678.569372
2209,46642.02,8.675409,4.034205e+05,802,5.813022,2.704186e+05,35615.036692,46394.490,46570.059928,68.916665,...,45831.052416,46356.3962,46570.059928,89.671000,46646.612797,46066.179603,46791.721095,45921.071305,46936.829394,45775.963006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,40069.94,30.778356,1.235268e+06,748,13.593420,5.455255e+05,30787.833940,40098.000,40093.539576,44.545002,...,39357.626248,40303.6484,40093.539576,86.166500,41079.939612,39527.357188,41468.085218,39139.211582,41856.230824,38751.065976
2440,39809.29,48.424642,1.932353e+06,1457,23.529697,9.391362e+05,30686.263272,40024.968,39904.039859,26.589907,...,39364.123304,40253.8756,39904.039859,92.273833,40977.699612,39530.051588,41339.611618,39168.139582,41701.523623,38806.227577
2441,39717.81,31.457964,1.254222e+06,1175,15.010662,5.985600e+05,30429.802905,39976.386,39779.886620,31.532532,...,39353.123305,40203.2212,39779.886620,103.202333,40870.909611,39535.532789,41204.753817,39201.688583,41538.598023,38867.844377
2442,39714.90,67.193329,2.673364e+06,1797,31.549799,1.255403e+06,30356.109563,39896.008,39736.562207,10.170264,...,39346.549677,40151.1976,39736.562207,115.296667,40748.530882,39553.864318,41047.197523,39255.197677,41345.864165,38956.531035


In [123]:
cash = 50
fee_rate = .00075
PT_test = PaperTrader(cash)
X_test_temp = X_test['close_price'].reset_index()
for i, pred in enumerate(y_pred_test):
    if pred:
        buy_amount = (cash*.5/X_test_temp['close_price'][i])
        # PT_test.free_cash  = PT_test.free_cash  - (buy_amount/(fee_rate))
        key = PT_test.buy('BTCUSD', X_test_temp['close_price'][i], buy_amount, fee_rate)
        try:
            PT_test.sell(key, X_test_temp['close_price'][i+1], buy_amount*(1-fee_rate), fee_rate)
        except:
            pass
print(f"{fee_rate*100}%")
paper_test = pd.DataFrame(PT_test.current_record())
print(PT_test.current_free_cash())
paper_test.to_csv('paper_trader_results.csv')

0.075%
45.7758686895485


In [75]:
data_test = pd.DataFrame(client.get_price_history_every_hour('BTCUSD').json()['candles'])
# pred_data, _, _ = preprocessing(data_test, price_offset = 1.000001, prediction = True, best_d_value = best_d_value, len_of_original_weights = len(weights))

AttributeError: 'Client' object has no attribute 'get_price_history_every_hour'

In [ ]:
client.get_historical_klines('BTCUSD', '1h', start_str = '2022-04-17', end_str=f'{datetime.now()}', limit=1000)